# Implementing Callbacks in TensorFlow using the MNIST Dataset

Here we are practicing classification MNIST which has items of handwriting -- the digits 0 through 9. We write an MNIST classifier that trains to 99% accuracy and stops once this threshold is achieved.

Some notes:
1. The network should succeed in less than 9 epochs.
2. When it reaches 99% or greater it should print out the string "Reached 99% accuracy so cancelling training!" and stop training.

In [7]:
import os
import tensorflow as tf
from tensorflow import keras

## Load and inspect the data

Begin by loading the data. A couple of things to notice:

- The file `mnist.npz` is already included in the current workspace under the `data` directory. By default the `load_data` from Keras accepts a path relative to `~/.keras/datasets` but in this case it is stored somewhere else, as a result of this, you need to specify the full path.

- `load_data` returns the train and test sets in the form of the tuples `(x_train, y_train), (x_test, y_test)` but in this exercise you will be needing only the train set so you can ignore the second tuple.

In [8]:
# Load the data

# Get current working directory
current_dir = os.getcwd()

# Discard test set
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
        
# Normalize pixel values
x_train = x_train / 255.0

Now take a look at the shape of the training data:

In [9]:
data_shape = x_train.shape

print(f"There are {data_shape[0]} examples with shape ({data_shape[1]}, {data_shape[2]})")

There are 60000 examples with shape (28, 28)


## Defining your callback

Now it is time to create your own custom callback. For this complete the `myCallback` class and the `on_epoch_end` method in the cell below. If you need some guidance on how to proceed, check out this [link](https://www.tensorflow.org/guide/keras/custom_callback).

In [10]:
# CLASS: myCallback

# Remember to inherit from the correct class
class myCallback(tf.keras.callbacks.Callback):
        # Define the correct function signature for on_epoch_end
        def on_epoch_end(self, epoch, logs={}):
            if (logs.get('accuracy') >= 0.99):
                print("\nReached 99% accuracy so cancelling training!") 
                
                # Stop training once the above condition is met
                self.model.stop_training = True

## Create and train your model

Now that you have defined your callback it is time to complete the `train_mnist` function below. 

**You must set your model to train for 10 epochs and the callback should fire before the 9th epoch for you to pass this assignment.**

**Hint:**
- Feel free to try the architecture for the neural network that you see fit but in case you need extra help you can check out an architecture that works pretty well at the end of this notebook.

In [11]:
# FUNCTION: train_mnist
def train_mnist(x_train, y_train):
    
    # Instantiate the callback class
    callbacks = myCallback()
    
    # Define the model
    model = tf.keras.models.Sequential([ 
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ]) 

    # Compile the model
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy']) 
    
    # Fit the model for 10 epochs adding the callbacks
    # and save the training history
    history = model.fit(x_train, y_train, epochs=10, callbacks=[callbacks])

    return history

Call the `train_mnist` passing in the appropiate parameters to get the training history:

In [12]:
hist = train_mnist(x_train, y_train)

Epoch 1/10
1875/1875 [==============================] - 7s 2ms/step - loss: 0.1999 - accuracy: 0.9419
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0812 - accuracy: 0.9752
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0532 - accuracy: 0.9836
Epoch 4/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0372 - accuracy: 0.9886
Epoch 5/10
1870/1875 [============================>.] - ETA: 0s - loss: 0.0274 - accuracy: 0.9911
Reached 99% accuracy so cancelling training!
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0274 - accuracy: 0.9912


If we see the message `Reached 99% accuracy so cancelling training!` printed out after less than 9 epochs it means your callback worked as expected. 